In [157]:
import pandas as pd
pd.set_option('display.max_columns', None)
import json
import math
import numpy as np

In [ ]:

language_normalization = {
    "APL": "APL",
    "Ada": "Ada",
    "Android": "Java",
    "AngularJS": "JavaScript",
    " AngularJS": "JavaScript",
    "Apex": "Apex",
    "Assembly": "Assembly",
    "Bash/Shell": "Bash",
    "Bash/Shell (all shells)": "Bash",
    "Bash/Shell/PowerShell": "Bash",
    "C": "C",
    "C#": "C#",
    "C++": "C++",
    "C++11": "C++",
    "Cobol": "COBOL",
    "COBOL": "COBOL",
    "Clojure": "Clojure",
    ' Clojure': "Clojure",
    "CSS": "HTML/CSS",
    "CoffeeScript": "CoffeeScript",
    "Common Lisp": "Lisp",
    "Crystal": "Crystal",
    "Dart": "Dart",
    "Delphi": "Delphi",
    "Delphi/Object Pascal": "Delphi",
    "Elixir": "Elixir",
    "Erlang": "Erlang",
    "F#": "F#",
    "Flow": "Flow",
    "Fortran": "Fortran",
    "GDScript": "GDScript",
    "Go": "Go",
    "Groovy": "Groovy",
    "Hack": "Hack",
    "Haskell": "Haskell",
    "HTML": "HTML/CSS",
    "HTML/CSS": "HTML/CSS",
    "Java": "Java",
    "JavaScript": "JavaScript",
    "Julia": "Julia",
    "Kotlin": "Kotlin",
    "LISP": "Lisp",
    "Lisp": "Lisp",
    "Lua": "Lua",
    "MATLAB": "Matlab",
    "Matlab": "Matlab",
    "MicroPython": "Python",
    "Nim": "Nim",
    "Node.js": "JavaScript",
    "OCaml": "OCaml",
    "Ocaml": "OCaml",
    "Objective-C": "Objective-C",
    "Perl": "Perl",
    "PHP": "PHP",
    "PowerShell": "Bash",
    "Prolog": "Prolog",
    "Python": "Python",
    "R": "R",
    "Raku": "Raku",
    "ReactJS": "JavaScript",
    "Redis": "Redis",
    "Ruby": "Ruby",
    "Rust": "Rust",
    "SAS": "SAS",
    "SQL": "SQL",
    "SQL Server": "SQL",
    "Scala": "Scala",
    "Shell": "Bash",
    "Smalltalk": "Smalltalk",
    "Solidity": "Solidity",
    "Spark": "Spark",
    "Swift": "Swift",
    "TypeScript": "TypeScript",
    "VB.NET": "VB.NET",
    "VBA": "VB.NET",
    "Visual Basic": "VB.NET",
    "Visual Basic (.Net)": "VB.NET",
    "Visual Basic 6": "VB.NET",
    "WebAssembly": "WebAssembly",
    "WordPress": "WordPress",
    "Wordpress": "WordPress",
    "Zig": "Zig",
    "iOS": "Swift"
}


valid_languages = {
    "Ada",
    "APL",
    "Apex",
    "Bash",
    "C",
    "C#",
    "C++",
    "COBOL",
    "CoffeeScript",
    "Clojure",
    "Crystal",
    "Dart",
    "Delphi",
    "Elixir",
    "Erlang",
    "F#",
    "Flow",
    "Fortran",
    "GDScript",
    "Go",
    "Groovy",
    "Hack",
    "Haskell",
    "HTML/CSS",
    "Java",
    "JavaScript",
    "Julia",
    "Kotlin",
    "Lisp",
    "Lua",
    "Matlab",
    "Nim",
    "OCaml",
    "Objective-C",
    "Perl",
    "PHP",
    "Prolog",
    "Python",
    "R",
    "Raku",
    "Ruby",
    "Rust",
    "Scala",
    "Smalltalk",
    "Solidity",
    "SQL",
    "Swift",
    "TypeScript",
    "VB.NET",
    "Visual Basic",
    "WebAssembly",
    "Zig"
}


def normalize_language(lang):
    return language_normalization.get(lang.strip(), lang.strip())



In [159]:
DATASETS_ROOT = '../../datasets'

# 2013

In [160]:
df_2013 = pd.read_csv(f'{DATASETS_ROOT}/2013 Stack Overflow Survey Responses.csv')

new_used_labels = df_2013.iloc[0, 57:70].tolist()
new_interested_labels = df_2013.iloc[0, 71:80].tolist()

for i, new_label in zip(range(57, 70), new_used_labels):
    df_2013.rename(columns={df_2013.columns[i]: new_label}, inplace=True)

for i, new_label in zip(range(71, 80), new_interested_labels):
    df_2013.rename(columns={df_2013.columns[i]: new_label}, inplace=True)

df_2013_cleaned = df_2013.drop(index=0).reset_index(drop=True)

used_lang_cols_named = new_used_labels[:-1]  # Exclude 'Other (please specify)'
interested_lang_cols_named = new_interested_labels

sample_data_cleaned = []
for _, row in df_2013_cleaned.iterrows():
    def safe_get(col):
        val = row.get(col)
        return None if pd.isna(val) or (isinstance(val, float) and math.isnan(val)) else val

    languages_used = [
        normalize_language(col) for col in used_lang_cols_named 
        if safe_get(col)
    ]
    languages_interested = [
        normalize_language(col) for col in interested_lang_cols_named 
        if safe_get(col)
    ]
    
    if len(languages_interested) == 0 and len(languages_used) == 0:
        continue

    sample_data_cleaned.append({
        "salary": safe_get("Including bonus, what is your annual compensation in USD?"),
        "languages_used": languages_used,
        "languages_interested": languages_interested,
        "country": safe_get("What Country or Region do you live in?"),
        "age": safe_get("How old are you?"),
        "year": 2013,
    })

with open(f'{DATASETS_ROOT}/clean/2013.json', 'w') as f:
    json.dump(sample_data_cleaned, f, indent=2)

/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/1006998086.py:1: DtypeWarning: Columns (46,48,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2013 = pd.read_csv(f'{DATASETS_ROOT}/2013 Stack Overflow Survey Responses.csv')


# 2014

In [161]:
df_2014 = pd.read_csv(f'{DATASETS_ROOT}/2014 Stack Overflow Survey Responses.csv')


used_labels_2014 = df_2014.iloc[0, 43:54].tolist()
interested_labels_2014 = df_2014.iloc[0, 55:67].tolist()

for i, new_label in zip(range(43, 54), used_labels_2014):
    df_2014.rename(columns={df_2014.columns[i]: new_label}, inplace=True)

for i, new_label in zip(range(55, 67), interested_labels_2014):
    df_2014.rename(columns={df_2014.columns[i]: new_label}, inplace=True)

df_2014_cleaned = df_2014.drop(index=0).reset_index(drop=True)

used_lang_cols_2014 = used_labels_2014
interested_lang_cols_2014 = interested_labels_2014

def normalize_language_filtered(lang):
    norm = language_normalization.get(lang.strip(), lang.strip())
    return norm if norm in valid_languages else None

# Helper to safely get values and coerce NaN to None
def safe_get(row, col):
    val = row.get(col)
    return None if pd.isna(val) or (isinstance(val, float) and pd.isna(val)) else val


cleaned_2014_data = []
for _, row in df_2014_cleaned.iterrows():
    used = [normalize_language_filtered(col) for col in used_lang_cols_2014 if safe_get(row, col)]
    interested = [normalize_language_filtered(col) for col in interested_lang_cols_2014 if safe_get(row, col)]

    used = [lang for lang in used if lang]
    interested = [lang for lang in interested if lang]

    cleaned_2014_data.append({
        "salary": safe_get(row, "Including bonus, what is your annual compensation in USD?"),
        "languages_used": used,
        "languages_interested": interested,
        "country": safe_get(row, "What Country do you live in?"),
        "age": safe_get(row, "How old are you?"),
        "year": 2014,
    })

import json
with open(f'{DATASETS_ROOT}/clean/2014.json', 'w') as f:
    json.dump(cleaned_2014_data, f, indent=2)

# 2015

In [162]:
invalid_languages = set()

In [163]:
df_2015 = pd.read_csv(f'{DATASETS_ROOT}/2015 Stack Overflow Survey Responses.csv')


used_lang_labels_2015 = df_2015.iloc[0, 9:51].tolist()
interested_lang_labels_2015 = df_2015.iloc[0, 52:94].tolist()

def strip_prefix(label):
    if isinstance(label, str):
        return label.split(":")[-1].strip()
    return None

used_lang_labels_clean = [strip_prefix(label) for label in used_lang_labels_2015 if strip_prefix(label)]
interested_lang_labels_clean = [strip_prefix(label) for label in interested_lang_labels_2015 if strip_prefix(label)]

used_lang_cols_2015 = used_lang_labels_clean[:-1]
interested_lang_cols_2015 = interested_lang_labels_clean[:-1]

used_lang_cols_2015, interested_lang_cols_2015


def normalize_language_filtered_2015(lang):
    norm = language_normalization.get(lang.strip(), lang.strip())
    
    if norm in valid_languages:
        return norm
    else:
        invalid_languages.add(lang)
        return None

column_mapping = {label: df_2015.columns[i+9] for i, label in enumerate(used_lang_cols_2015)}
interested_column_mapping = {label: df_2015.columns[i+52] for i, label in enumerate(interested_lang_cols_2015)}

df_2015_cleaned = df_2015.drop(index=0).reset_index(drop=True)

cleaned_2015_data = []
for _, row in df_2015_cleaned.iterrows():
    used = [
        normalize_language_filtered_2015(lang)
        for lang in used_lang_cols_2015
        if pd.notna(row.get(column_mapping[lang])) and str(row.get(column_mapping[lang])).strip()
    ]
    interested = [
        normalize_language_filtered_2015(lang)
        for lang in interested_lang_cols_2015
        if pd.notna(row.get(interested_column_mapping[lang])) and str(row.get(interested_column_mapping[lang])).strip()
    ]

    used = [lang for lang in used if lang]
    interested = [lang for lang in interested if lang]

    cleaned_2015_data.append({
        "salary": safe_get(row, "Unnamed: 105"),
        "languages_used": used,
        "languages_interested": interested,
        "country": safe_get(row, "Unnamed: 0"),
        "age": safe_get(row, "Unnamed: 1"),
        "year": 2015,
    })

with open(f'{DATASETS_ROOT}/clean/2015.json', 'w') as f:
    json.dump(cleaned_2015_data, f, indent=2)

/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/3947357209.py:1: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2015 = pd.read_csv(f'{DATASETS_ROOT}/2015 Stack Overflow Survey Responses.csv')


# 2016

In [164]:
df_2016 = pd.read_csv(f'{DATASETS_ROOT}/2016 Stack Overflow Survey Results/2016 Stack Overflow Survey Responses.csv')

def normalize_lang_list_2016(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2016_data = []
for _, row in df_2016.iterrows():
    cleaned_2016_data.append({
        "salary": safe_get(row, "salary_range"),
        "languages_used": normalize_lang_list_2016(row.get("tech_do")),
        "languages_interested": normalize_lang_list_2016(row.get("tech_want")),
        "country": safe_get(row, "country"),
        "age": safe_get(row, "age_midpoint"),
        "year": 2016,
    })

with open(f'{DATASETS_ROOT}/clean/2016.json', 'w') as f:
    json.dump(cleaned_2016_data, f, indent=2)

# 2017

In [165]:
df_2017 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2017/survey_results_public.csv')

def normalize_lang_list(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2017_data = []
for _, row in df_2017.iterrows():
    cleaned_2017_data.append({
        "salary": safe_get(row, "Salary"),
        "languages_used": normalize_lang_list(row.get("HaveWorkedLanguage")),
        "languages_interested": normalize_lang_list(row.get("WantWorkLanguage")),
        "country": safe_get(row, "Country"),
        "age": None,  # Not available in this dataset
        "year": 2017,
    })

with open(f'{DATASETS_ROOT}/clean/2017.json', 'w') as f:
    json.dump(cleaned_2017_data, f, indent=2)


# 2018

In [166]:
df_2018 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2018/survey_results_public.csv')

def normalize_lang_list_2018(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2018_data = []
for _, row in df_2018.iterrows():
    cleaned_2018_data.append({
        "salary": safe_get(row, "ConvertedSalary"),
        "languages_used": normalize_lang_list_2018(row.get("LanguageWorkedWith")),
        "languages_interested": normalize_lang_list_2018(row.get("LanguageDesireNextYear")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2018,
    })

with open(f'{DATASETS_ROOT}/clean/2018.json', 'w') as f:
    json.dump(cleaned_2018_data, f, indent=2)

/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/108227214.py:1: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2018 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2018/survey_results_public.csv')


# 2019

In [167]:
df_2019 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2019/survey_results_public.csv')

def normalize_lang_list_2019(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2019_data = []
for _, row in df_2019.iterrows():
    cleaned_2019_data.append({
        "salary": safe_get(row, "ConvertedComp"),
        "languages_used": normalize_lang_list_2019(row.get("LanguageWorkedWith")),
        "languages_interested": normalize_lang_list_2019(row.get("LanguageDesireNextYear")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2019,
    })

with open(f'{DATASETS_ROOT}/clean/2019.json', 'w') as f:
    json.dump(cleaned_2019_data, f, indent=2)

# 2020

In [168]:
df_2020 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2020/survey_results_public.csv')

def normalize_lang_list_2020(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2020_data = []
for _, row in df_2020.iterrows():
    cleaned_2020_data.append({
        "salary": safe_get(row, "ConvertedComp"),
        "languages_used": normalize_lang_list_2020(row.get("LanguageWorkedWith")),
        "languages_interested": normalize_lang_list_2020(row.get("LanguageDesireNextYear")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2020,
    })

with open(f'{DATASETS_ROOT}/clean/2020.json', 'w') as f:
    json.dump(cleaned_2020_data, f, indent=2)

# 2021

In [169]:
df_2021 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2021/survey_results_public.csv')

def normalize_lang_list_2021(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2021_data = []
for _, row in df_2021.iterrows():
    cleaned_2021_data.append({
        "salary": safe_get(row, "ConvertedCompYearly"),
        "languages_used": normalize_lang_list_2021(row.get("LanguageHaveWorkedWith")),
        "languages_interested": normalize_lang_list_2021(row.get("LanguageWantToWorkWith")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2021,
    })

with open(f'{DATASETS_ROOT}/clean/2021.json', 'w') as f:
    json.dump(cleaned_2021_data, f, indent=2)


# 2022

In [170]:
df_2022 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2022/survey_results_public.csv')

def normalize_lang_list_2022(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2022_data = []
for _, row in df_2022.iterrows():
    cleaned_2022_data.append({
        "salary": safe_get(row, "ConvertedCompYearly"),
        "languages_used": normalize_lang_list_2022(row.get("LanguageHaveWorkedWith")),
        "languages_interested": normalize_lang_list_2022(row.get("LanguageWantToWorkWith")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2022,
    })

with open(f'{DATASETS_ROOT}/clean/2022.json', 'w') as f:
    json.dump(cleaned_2022_data, f, indent=2)


# 2023

In [171]:
df_2023 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2023/survey_results_public.csv')


def normalize_lang_list_2023(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2023_data = []
for _, row in df_2023.iterrows():
    cleaned_2023_data.append({
        "salary": safe_get(row, "ConvertedCompYearly"),
        "languages_used": normalize_lang_list_2023(row.get("LanguageHaveWorkedWith")),
        "languages_interested": normalize_lang_list_2023(row.get("LanguageWantToWorkWith")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2023,
    })

with open(f'{DATASETS_ROOT}/clean/2023.json', 'w') as f:
    json.dump(cleaned_2023_data, f, indent=2)

# 2024

In [172]:
df_2024 = pd.read_csv(f'{DATASETS_ROOT}/stack-overflow-developer-survey-2024/survey_results_public.csv')


def normalize_lang_list_2024(lang_string):
    if pd.isna(lang_string):
        return []
    langs = [normalize_language_filtered_2015(lang) for lang in lang_string.split(';')]
    return [lang for lang in langs if lang]

cleaned_2024_data = []
for _, row in df_2024.iterrows():
    cleaned_2024_data.append({
        "salary": safe_get(row, "ConvertedCompYearly"),
        "languages_used": normalize_lang_list_2024(row.get("LanguageHaveWorkedWith")),
        "languages_interested": normalize_lang_list_2024(row.get("LanguageWantToWorkWith")),
        "country": safe_get(row, "Country"),
        "age": safe_get(row, "Age"),
        "year": 2024,
    })

with open(f'{DATASETS_ROOT}/clean/2024.json', 'w') as f:
    json.dump(cleaned_2024_data, f, indent=2)


### Invalid languages

Let's see what gets removed

In [173]:
invalid_languages

{' AngularJS',
 ' Arduino / Raspberry Pi',
 ' Cassandra',
 ' Clojure',
 ' Cloud (AWS, GAE, Azure, etc.)',
 ' Cordova',
 ' Hadoop',
 ' LAMP',
 ' MongoDB',
 ' Redis',
 ' Salesforce',
 ' SharePoint',
 ' Spark',
 ' Windows Phone',
 ' WordPress',
 'AngularJS',
 'Arduino',
 'Arduino / Raspberry Pi',
 'Assembly',
 'CSS',
 'Cassandra',
 'Clojure',
 'Cloud',
 'Cloud (AWS, GAE, Azure, etc.)',
 'Cobol',
 'Cordova',
 'Flow',
 'HTML',
 'HTML/CSS',
 'Hadoop',
 'LAMP',
 'MongoDB',
 'Other(s):',
 'Redis',
 'SAS',
 'Salesforce',
 'SharePoint',
 'Sharepoint',
 'Spark',
 'Windows Phone',
 'WordPress',
 'Wordpress',
 'Zephyr'}

# Contruct final dataset for visualization

In [174]:
import glob
import json

all_data =  []

for path in glob.glob(f'{DATASETS_ROOT}/clean/*.json'):
    with open(path) as f:
        all_data += json.load(f)


df = pd.DataFrame(all_data)
df

,salary,languages_used,languages_interested,country,age,year
0,None,[],[],None,None,2022
1,None,"[JavaScript, TypeScript]","[Rust, TypeScript]",Canada,None,2022
2,40205.0,"[C#, C++, JavaScript, Python]","[C#, C++, JavaScript, TypeScript]",United Kingdom of Great Britain and Northern I...,25-34 years old,2022
3,215232.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",Israel,35-44 years old,2022
4,None,"[C#, JavaScript, SQL, Swift, TypeScript]","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",United States of America,25-34 years old,2022
...,...,...,...,...,...,...
712645,160500.0,"[Kotlin, SQL]",[],United States of America,25-34 years old,2021
712646,3960.0,[],[],Benin,18-24 years old,2021
712647,90000.0,"[Groovy, Java, Python]","[Java, Python]",United States of America,25-34 years old,2021
712648,816816.0,"[Bash, JavaScript, JavaScript, Python]","[Go, Rust]",Canada,25-34 years old,2021


## Unify age ranges

In [175]:
def map_age(age):
    if pd.isna(age):
        return 'Prefer not to say'

    if isinstance(age, str):
        age = age.strip()
        if age in [
            'Under 18 years old', '18-24 years old', '25-34 years old',
            '35-44 years old', '45-54 years old', '55-64 years old',
            '65 years or older', 'Prefer not to say'
        ]:
            return age
        if age == '25 - 34 years old':
            return '25-34 years old'
        if age == '35 - 44 years old':
            return '35-44 years old'
        if age == '18 - 24 years old':
            return '18-24 years old'
        if age == '45 - 54 years old':
            return '45-54 years old'
        if age == '55 - 64 years old':
            return '55-64 years old'
        if age in ['Prefer not to disclose']:
            return 'Prefer not to say'
        return 'Prefer not to say'

    try:
        age_float = float(age)
    except:
        return 'Prefer not to say'

    if age_float < 18:
        return 'Under 18 years old'
    elif 18 <= age_float <= 24:
        return '18-24 years old'
    elif 25 <= age_float <= 34:
        return '25-34 years old'
    elif 35 <= age_float <= 44:
        return '35-44 years old'
    elif 45 <= age_float <= 54:
        return '45-54 years old'
    elif 55 <= age_float <= 64:
        return '55-64 years old'
    elif age_float >= 65:
        return '65 years or older'
    else:
        return 'Prefer not to say'

df['age'] = df['age'].apply(map_age)
print(df['age'].unique())
df

['Prefer not to say' '25-34 years old' '35-44 years old'
 'Under 18 years old' '18-24 years old' '45-54 years old'
 '55-64 years old' '65 years or older']


,salary,languages_used,languages_interested,country,age,year
0,None,[],[],None,Prefer not to say,2022
1,None,"[JavaScript, TypeScript]","[Rust, TypeScript]",Canada,Prefer not to say,2022
2,40205.0,"[C#, C++, JavaScript, Python]","[C#, C++, JavaScript, TypeScript]",United Kingdom of Great Britain and Northern I...,25-34 years old,2022
3,215232.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",Israel,35-44 years old,2022
4,None,"[C#, JavaScript, SQL, Swift, TypeScript]","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",United States of America,25-34 years old,2022
...,...,...,...,...,...,...
712645,160500.0,"[Kotlin, SQL]",[],United States of America,25-34 years old,2021
712646,3960.0,[],[],Benin,18-24 years old,2021
712647,90000.0,"[Groovy, Java, Python]","[Java, Python]",United States of America,25-34 years old,2021
712648,816816.0,"[Bash, JavaScript, JavaScript, Python]","[Go, Rust]",Canada,25-34 years old,2021


## Salaries are only used to determine bubble size

The reason is that they are missing values and mostly ranges.

In [176]:
salary_str = df['salary'].astype(str)

non_numeric_salaries = salary_str[~salary_str.str.replace('.', '', 1).str.isnumeric()]

unique_salary_ranges = non_numeric_salaries.unique()
print(unique_salary_ranges)

['None' '$20,000 - $40,000' 'Student / Unemployed' '<$20,000'
 'Rather not say' '$80,000 - $100,000' '$60,000 - $80,000'
 '$40,000 - $60,000' '>$140,000' '$100,000 - $120,000'
 '$120,000 - $140,000' 'Less than $20,000' 'Unemployed'
 '$140,000 - $160,000' 'More than $160,000' '$40,000 - $50,000'
 'Less than $10,000' 'More than $200,000' '$10,000 - $20,000'
 '$90,000 - $100,000' '$30,000 - $40,000' '$20,000 - $30,000'
 '$70,000 - $80,000' '$80,000 - $90,000' '$50,000 - $60,000'
 '$60,000 - $70,000' '$140,000 - $150,000' '$130,000 - $140,000'
 '$100,000 - $110,000' '$110,000 - $120,000' '$160,000 - $170,000'
 '$180,000 - $190,000' 'Other (please specify)' '$120,000 - $130,000'
 '$150,000 - $160,000' '$190,000 - $200,000' '$170,000 - $180,000']


In [177]:
salary_range_to_midpoint = {
    # Null-like
    'None': None,
    'Rather not say': None,
    'Student / Unemployed': None,
    'Unemployed': None,
    'Other (please specify)': None,

    # Low and high bounds (we'll take the visible number)
    '<$20,000': 20000,
    'Less than $20,000': 20000,
    'Less than $10,000': 10000,
    'More than $160,000': 160000,
    'More than $200,000': 200000,
    '>$140,000': 140000,

    # Ranges _> midpoints
    '$10,000 - $20,000': 15000,
    '$20,000 - $30,000': 25000,
    '$20,000 - $40,000': 30000,
    '$30,000 - $40,000': 35000,
    '$40,000 - $50,000': 45000,
    '$40,000 - $60,000': 50000,
    '$50,000 - $60,000': 55000,
    '$60,000 - $70,000': 65000,
    '$60,000 - $80,000': 70000,
    '$70,000 - $80,000': 75000,
    '$80,000 - $90,000': 85000,
    '$80,000 - $100,000': 90000,
    '$90,000 - $100,000': 95000,
    '$100,000 - $110,000': 105000,
    '$100,000 - $120,000': 110000,
    '$110,000 - $120,000': 115000,
    '$120,000 - $130,000': 125000,
    '$120,000 - $140,000': 130000,
    '$130,000 - $140,000': 135000,
    '$140,000 - $150,000': 145000,
    '$140,000 - $160,000': 150000,
    '$150,000 - $160,000': 155000,
    '$160,000 - $170,000': 165000,
    '$170,000 - $180,000': 175000,
    '$180,000 - $190,000': 185000,
    '$190,000 - $200,000': 195000,
}

def parse_salary(s):
    if pd.isna(s):
        return np.nan
    try:
        return float(s)
    except (ValueError, TypeError):
        return salary_range_to_midpoint.get(str(s).strip(), np.nan)

df['salary'] = df['salary'].apply(parse_salary).astype(float)

df

,salary,languages_used,languages_interested,country,age,year
0,NaN,[],[],None,Prefer not to say,2022
1,NaN,"[JavaScript, TypeScript]","[Rust, TypeScript]",Canada,Prefer not to say,2022
2,40205.0,"[C#, C++, JavaScript, Python]","[C#, C++, JavaScript, TypeScript]",United Kingdom of Great Britain and Northern I...,25-34 years old,2022
3,215232.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",Israel,35-44 years old,2022
4,NaN,"[C#, JavaScript, SQL, Swift, TypeScript]","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",United States of America,25-34 years old,2022
...,...,...,...,...,...,...
712645,160500.0,"[Kotlin, SQL]",[],United States of America,25-34 years old,2021
712646,3960.0,[],[],Benin,18-24 years old,2021
712647,90000.0,"[Groovy, Java, Python]","[Java, Python]",United States of America,25-34 years old,2021
712648,816816.0,"[Bash, JavaScript, JavaScript, Python]","[Go, Rust]",Canada,25-34 years old,2021


In [178]:
country_mapping = {
    None: "Prefer not to say",
    "I prefer not to say": "Prefer not to say",
    
    # United States
    "United States of America": "United States",
    "United States": "United States",
    "Virgin Islands (USA)": "United States",
    "American Samoa": "United States",

    # United Kingdom
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "United Kingdom": "United Kingdom",
    "UK": "United Kingdom",
    "Isle of Man": "United Kingdom",

    # Russia
    "Russian Federation": "Russia",
    "Russia": "Russia",

    # South Korea
    "Republic of Korea": "South Korea",
    "Korea South": "South Korea",
    "South Korea": "South Korea",

    # North Korea
    "Democratic People's Republic of Korea": "North Korea",
    "Korea North": "North Korea",
    "North Korea": "North Korea",

    # Iran
    "Iran, Islamic Republic of...": "Iran",
    "Iran": "Iran",

    # Vietnam
    "Viet Nam": "Vietnam",

    # Ivory Coast
    "Côte d'Ivoire": "Ivory Coast",
    "Ivory Coast": "Ivory Coast",
    "Ivory Coast (Cote D'Ivoire)": "Ivory Coast",

    # Macedonia
    "The former Yugoslav Republic of Macedonia": "North Macedonia",
    "Republic of North Macedonia": "North Macedonia",
    "Macedonia [FYROM]": "North Macedonia",
    "Macedonia": "North Macedonia",

    # Bosnia
    "Bosnia and Herzegovina": "Bosnia and Herzegovina",
    "Bosnia Herzegovina": "Bosnia and Herzegovina",
    "Bosnia-Herzegovina": "Bosnia and Herzegovina",

    # Venezuela
    "Venezuela, Bolivarian Republic of...": "Venezuela",

    # Myanmar
    "Myanmar [Burma]": "Myanmar",
    "Myanmar, {Burma}": "Myanmar",

    # Misc catchalls
    "Other": "Other",
    "Other Country (Not Listed Above)": "Other",
    "Other (please specify)": "Other",
    "Other Europe": "Other",
    "Other Asia": "Other",
    "South America": "Other",
    "Australasia": "Other",
    "Middle East": "Other",
    "Central America": "Other",
    "North America (Other)": "Other",
    "Africa": "Other",
    "Antarctica": "Other",
    "Nomadic": "Other"
}


df['country'] = df['country'].map(lambda x: country_mapping.get(x, x))

df

,salary,languages_used,languages_interested,country,age,year
0,NaN,[],[],Prefer not to say,Prefer not to say,2022
1,NaN,"[JavaScript, TypeScript]","[Rust, TypeScript]",Canada,Prefer not to say,2022
2,40205.0,"[C#, C++, JavaScript, Python]","[C#, C++, JavaScript, TypeScript]",United Kingdom,25-34 years old,2022
3,215232.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",Israel,35-44 years old,2022
4,NaN,"[C#, JavaScript, SQL, Swift, TypeScript]","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",United States,25-34 years old,2022
...,...,...,...,...,...,...
712645,160500.0,"[Kotlin, SQL]",[],United States,25-34 years old,2021
712646,3960.0,[],[],Benin,18-24 years old,2021
712647,90000.0,"[Groovy, Java, Python]","[Java, Python]",United States,25-34 years old,2021
712648,816816.0,"[Bash, JavaScript, JavaScript, Python]","[Go, Rust]",Canada,25-34 years old,2021


## Drop the rows where both languages columns are empy

In [179]:
df = df[~((df['languages_used'].apply(len) == 0) & (df['languages_interested'].apply(len) == 0))]

df

,salary,languages_used,languages_interested,country,age,year
1,NaN,"[JavaScript, TypeScript]","[Rust, TypeScript]",Canada,Prefer not to say,2022
2,40205.0,"[C#, C++, JavaScript, Python]","[C#, C++, JavaScript, TypeScript]",United Kingdom,25-34 years old,2022
3,215232.0,"[C#, JavaScript, SQL, TypeScript]","[C#, SQL, TypeScript]",Israel,35-44 years old,2022
4,NaN,"[C#, JavaScript, SQL, Swift, TypeScript]","[C#, Elixir, F#, Go, JavaScript, Rust, TypeScr...",United States,25-34 years old,2022
5,NaN,"[C++, Lua]",[Lua],Germany,25-34 years old,2022
...,...,...,...,...,...,...
712644,18326.0,"[Java, JavaScript, Kotlin, Objective-C, TypeSc...",[Kotlin],Brazil,25-34 years old,2021
712645,160500.0,"[Kotlin, SQL]",[],United States,25-34 years old,2021
712647,90000.0,"[Groovy, Java, Python]","[Java, Python]",United States,25-34 years old,2021
712648,816816.0,"[Bash, JavaScript, JavaScript, Python]","[Go, Rust]",Canada,25-34 years old,2021


## We de-duplicate the language columns

In [180]:
def normalize_language_list(lst):
    return [lang if lang not in {'jQuery', 'JQuery'} else 'JavaScript' for lang in lst]

df['languages_used'] = df['languages_used'].apply(normalize_language_list)
df['languages_interested'] = df['languages_interested'].apply(normalize_language_list)


df['languages_used'] = df['languages_used'].apply(lambda lst: list(set(lst)))
df['languages_interested'] = df['languages_interested'].apply(lambda lst: list(set(lst)))

df

/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/52294962.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['languages_used'] = df['languages_used'].apply(normalize_language_list)
/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/52294962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['languages_interested'] = df['languages_interested'].apply(normalize_language_list)
/var/folders/nt/t7ckgjg554g17zn445f3ccbc0000gn/T/ipykernel_25550/52294962.py:8: SettingWithCopyWar

,salary,languages_used,languages_interested,country,age,year
1,NaN,"[JavaScript, TypeScript]","[TypeScript, Rust]",Canada,Prefer not to say,2022
2,40205.0,"[C++, Python, JavaScript, C#]","[C++, JavaScript, TypeScript, C#]",United Kingdom,25-34 years old,2022
3,215232.0,"[JavaScript, TypeScript, C#, SQL]","[TypeScript, C#, SQL]",Israel,35-44 years old,2022
4,NaN,"[JavaScript, TypeScript, C#, SQL, Swift]","[JavaScript, TypeScript, C#, Go, Rust, Elixir,...",United States,25-34 years old,2022
5,NaN,"[C++, Lua]",[Lua],Germany,25-34 years old,2022
...,...,...,...,...,...,...
712644,18326.0,"[JavaScript, TypeScript, Java, Kotlin, Objecti...",[Kotlin],Brazil,25-34 years old,2021
712645,160500.0,"[Kotlin, SQL]",[],United States,25-34 years old,2021
712647,90000.0,"[Java, Python, Groovy]","[Java, Python]",United States,25-34 years old,2021
712648,816816.0,"[Python, JavaScript, Bash]","[Go, Rust]",Canada,25-34 years old,2021


## Now we create two datasets one is for used and one is for interestes

In [181]:
df_languages_used = df.explode('languages_used').dropna(subset=['languages_used']).copy()
df_languages_used = df_languages_used.rename(columns={'languages_used': 'language'})
df_languages_used = df_languages_used[['salary', 'language', 'country', 'age', 'year']]

df_languages_used

,salary,language,country,age,year
1,NaN,JavaScript,Canada,Prefer not to say,2022
1,NaN,TypeScript,Canada,Prefer not to say,2022
2,40205.0,C++,United Kingdom,25-34 years old,2022
2,40205.0,Python,United Kingdom,25-34 years old,2022
2,40205.0,JavaScript,United Kingdom,25-34 years old,2022
...,...,...,...,...,...
712648,816816.0,Bash,Canada,25-34 years old,2021
712649,21168.0,Elixir,Brazil,18-24 years old,2021
712649,21168.0,Java,Brazil,18-24 years old,2021
712649,21168.0,Delphi,Brazil,18-24 years old,2021


In [182]:
df_languages_interested = df.explode('languages_interested').dropna(subset=['languages_interested']).copy()
df_languages_interested = df_languages_interested.rename(columns={'languages_interested': 'language'})
df_languages_interested = df_languages_interested[['salary', 'language', 'country', 'age', 'year']]
non_languages = {"MongoDB", "Redis", "WinRT"}
df_languages_interested = df_languages_interested[~df_languages_interested['language'].isin(non_languages)]

df_languages_interested

,salary,language,country,age,year
1,NaN,TypeScript,Canada,Prefer not to say,2022
1,NaN,Rust,Canada,Prefer not to say,2022
2,40205.0,C++,United Kingdom,25-34 years old,2022
2,40205.0,JavaScript,United Kingdom,25-34 years old,2022
2,40205.0,TypeScript,United Kingdom,25-34 years old,2022
...,...,...,...,...,...
712649,21168.0,TypeScript,Brazil,18-24 years old,2021
712649,21168.0,PHP,Brazil,18-24 years old,2021
712649,21168.0,Java,Brazil,18-24 years old,2021
712649,21168.0,SQL,Brazil,18-24 years old,2021


In [183]:
df_languages_used['language'].value_counts() / len(df_languages_used)

language
JavaScript      0.152369
SQL             0.120628
Python          0.098880
Java            0.087386
C#              0.070953
Bash            0.069622
TypeScript      0.056980
PHP             0.056565
C++             0.054178
C               0.046056
Go              0.020618
Ruby            0.017637
VB.NET          0.015213
Kotlin          0.014717
Rust            0.014491
Swift           0.014375
R               0.011232
Objective-C     0.010249
Dart            0.008314
Matlab          0.007644
Scala           0.007322
Perl            0.005923
Lua             0.005697
Groovy          0.005217
Haskell         0.004118
Delphi          0.003617
Elixir          0.002858
CoffeeScript    0.002264
F#              0.002212
Julia           0.001790
Erlang          0.001678
Lisp            0.001644
GDScript        0.001051
Solidity        0.001031
Fortran         0.000778
OCaml           0.000712
Crystal         0.000525
Zig             0.000506
APL             0.000459
Prolog          

In [184]:
df_languages_used[df_languages_used['year'] == 2013]['language'].value_counts()

language
JavaScript     5149
SQL            4593
C#             3027
Java           3020
PHP            2324
Python         1880
C++            1696
Objective-C     955
Ruby            794
Name: count, dtype: int64

In [185]:
df_languages_interested['language'].value_counts()  / len(df_languages_interested)

language
JavaScript      0.122617
Python          0.107831
SQL             0.090228
TypeScript      0.071478
Java            0.063355
C#              0.063207
Bash            0.052628
Go              0.051310
C++             0.049757
Rust            0.047274
PHP             0.032872
C               0.032292
Kotlin          0.030684
Swift           0.026749
Ruby            0.019940
R               0.015532
Dart            0.014485
Scala           0.014288
Haskell         0.011946
F#              0.008777
Elixir          0.007923
Objective-C     0.007575
VB.NET          0.005616
Lua             0.005586
Julia           0.004909
Erlang          0.004677
Perl            0.004256
Matlab          0.003872
WebAssembly     0.003559
Zig             0.003093
Groovy          0.003034
CoffeeScript    0.003008
Lisp            0.002682
Delphi          0.002617
Solidity        0.002176
OCaml           0.001706
GDScript        0.001363
Crystal         0.000972
APL             0.000667
Fortran         

In [186]:
df_languages_interested[df_languages_interested['year'] == 2013]['language'].value_counts()

language
C++             1240
CoffeeScript    1018
Haskell          883
TypeScript       615
F#               608
Dart             411
Name: count, dtype: int64

## Append the group memberships

In [187]:
set([*df_languages_interested['language'].unique(), *df_languages_used['language'].unique()])

{'APL',
 'Ada',
 'Apex',
 'Bash',
 'C',
 'C#',
 'C++',
 'COBOL',
 'CoffeeScript',
 'Crystal',
 'Dart',
 'Delphi',
 'Elixir',
 'Erlang',
 'F#',
 'Fortran',
 'GDScript',
 'Go',
 'Groovy',
 'Hack',
 'Haskell',
 'Java',
 'JavaScript',
 'Julia',
 'Kotlin',
 'Lisp',
 'Lua',
 'Matlab',
 'Nim',
 'OCaml',
 'Objective-C',
 'PHP',
 'Perl',
 'Prolog',
 'Python',
 'R',
 'Raku',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Smalltalk',
 'Solidity',
 'Swift',
 'TypeScript',
 'VB.NET',
 'WebAssembly',
 'Zig'}

In [206]:
# 1) Compiled vs. Interpreted
compiled_vs_interpreted = {
    "Ada":           "compiled",
    "APL":           "interpreted",
    "Apex":          "compiled",
    "Bash":          "interpreted",
    "C":             "compiled",
    "C#":            "compiled",
    "C++":           "compiled",
    "COBOL":         "compiled",
    "CoffeeScript":  "interpreted",
    "Crystal":       "compiled",
    "Dart":          "compiled",
    "Delphi":        "compiled",
    "Elixir":        "compiled",
    "Erlang":        "compiled",
    "F#":            "compiled",
    "Fortran":       "compiled",
    "GDScript":      "interpreted",
    "Go":            "compiled",
    "Groovy":        "compiled",
    "Hack":          "compiled",
    "Haskell":       "compiled",
    "HTML/CSS":      "interpreted",
    "Java":          "compiled",
    "JavaScript":    "interpreted",
    "Julia":         "compiled",
    "Kotlin":        "compiled",
    "Lisp":          "interpreted",
    "Lua":           "interpreted",
    "Matlab":        "interpreted",
    "Nim":           "compiled",
    "OCaml":         "compiled",
    "Objective-C":   "compiled",
    "Perl":          "interpreted",
    "PHP":           "interpreted",
    "PowerShell":    "interpreted",
    "Prolog":        "interpreted",
    "Python":        "interpreted",
    "R":             "interpreted",
    "Raku":          "interpreted",
    "Ruby":          "interpreted",
    "Rust":          "compiled",
    "SAS":           "interpreted",
    "Scala":         "compiled",
    "Smalltalk":     "interpreted",
    "Solidity":      "compiled",
    "SQL":           "interpreted",
    "Swift":         "compiled",
    "TypeScript":    "compiled",
    "VB.NET":        "compiled",
    "Visual Basic":  "compiled",
    "WebAssembly":   "compiled",
    "Zig":           "compiled",
}


# 2) Memory Management Model
#    - 'manual'  = manual allocation / deallocation (or ARC-like)
#    - 'gc'      = garbage-collected
#    - 'na'      = not-applicable or external 
memory_management = {
    "Ada":           "manual",
    "APL":           "gc",
    "Apex":          "gc",
    "Bash":          "na",
    "C":             "manual",
    "C#":            "gc",
    "C++":           "manual",
    "COBOL":         "manual",
    "CoffeeScript":  "gc",
    "Crystal":       "gc",
    "Dart":          "gc",
    "Delphi":        "manual",
    "Elixir":        "gc",
    "Erlang":        "gc",
    "F#":            "gc",
    "Fortran":       "manual",
    "GDScript":      "gc",
    "Go":            "gc",
    "Groovy":        "gc",
    "Hack":          "gc",
    "Haskell":       "gc",
    "HTML/CSS":      "na",
    "Java":          "gc",
    "JavaScript":    "gc",
    "Julia":         "gc",
    "Kotlin":        "gc",
    "Lisp":          "gc",
    "Lua":           "gc",
    "Matlab":        "gc",
    "Nim":           "manual",
    "OCaml":         "gc",
    "Objective-C":   "manual",
    "Perl":          "gc",
    "PHP":           "gc",
    "PowerShell":    "gc",
    "Prolog":        "gc",
    "Python":        "gc",
    "R":             "gc",
    "Raku":          "gc",
    "Ruby":          "gc",
    "Rust":          "manual",
    "SAS":           "na",
    "Scala":         "gc",
    "Smalltalk":     "gc",
    "Solidity":      "manual",
    "SQL":           "na",
    "Swift":         "manual",
    "TypeScript":    "gc",
    "VB.NET":        "gc",
    "Visual Basic":  "gc",
    "WebAssembly":   "manual",
    "Zig":           "manual",
}


df_languages_interested['execution_model'] = df_languages_interested['language'].map(compiled_vs_interpreted)
df_languages_interested['memory_management'] = df_languages_interested['language'].map(memory_management)

df_languages_used['execution_model'] = df_languages_used['language'].map(compiled_vs_interpreted)
df_languages_used['memory_management'] = df_languages_used['language'].map(memory_management)


In [207]:
df_languages_interested

,salary,language,country,age,year,execution_model,memory_management
1,NaN,TypeScript,Canada,Prefer not to say,2022,compiled,gc
1,NaN,Rust,Canada,Prefer not to say,2022,compiled,manual
2,40205.0,C++,United Kingdom,25-34 years old,2022,compiled,manual
2,40205.0,JavaScript,United Kingdom,25-34 years old,2022,interpreted,gc
2,40205.0,TypeScript,United Kingdom,25-34 years old,2022,compiled,gc
...,...,...,...,...,...,...,...
712649,21168.0,TypeScript,Brazil,18-24 years old,2021,compiled,gc
712649,21168.0,PHP,Brazil,18-24 years old,2021,interpreted,gc
712649,21168.0,Java,Brazil,18-24 years old,2021,compiled,gc
712649,21168.0,SQL,Brazil,18-24 years old,2021,interpreted,na


## Finally we combine these to a single final dataset

In [208]:
df_languages_used_copy = df_languages_used.copy()
df_languages_interested_copy = df_languages_interested.copy()

df_languages_used_copy['discriminator'] = 'used'
df_languages_interested_copy['discriminator'] = 'interested'

df_languages_used_copy['execution_model'] = df_languages_used_copy['language'].map(compiled_vs_interpreted)
df_languages_used_copy['memory_management'] = df_languages_used_copy['language'].map(memory_management)

df_languages_all = pd.concat([df_languages_used_copy, df_languages_interested_copy], ignore_index=True)
df_languages_all

,salary,language,country,age,year,execution_model,memory_management,discriminator
0,NaN,JavaScript,Canada,Prefer not to say,2022,interpreted,gc,used
1,NaN,TypeScript,Canada,Prefer not to say,2022,compiled,gc,used
2,40205.0,C++,United Kingdom,25-34 years old,2022,compiled,manual,used
3,40205.0,Python,United Kingdom,25-34 years old,2022,interpreted,gc,used
4,40205.0,JavaScript,United Kingdom,25-34 years old,2022,interpreted,gc,used
...,...,...,...,...,...,...,...,...
5128189,21168.0,TypeScript,Brazil,18-24 years old,2021,compiled,gc,interested
5128190,21168.0,PHP,Brazil,18-24 years old,2021,interpreted,gc,interested
5128191,21168.0,Java,Brazil,18-24 years old,2021,compiled,gc,interested
5128192,21168.0,SQL,Brazil,18-24 years old,2021,interpreted,na,interested


In [209]:
df_languages_all[df_languages_all['language'] == 'TypeScript']['year'].unique()

array([2022, 2018, 2019, 2023, 2024, 2020, 2017, 2021, 2013])

In [210]:
df_languages_all.to_csv(f'{DATASETS_ROOT}/clean/final.csv', index=False)

# Pre-filtered datasets

The resulting 300mb dataset is a bit heavy for a client software. Let's create a dataset with aggregated data.


In the years 2013 and 2014 we run into issues that some languages results into having `used=0.0` and other `interest=0.0`. To overcome this, we use **temporal interpolation**

In [211]:
def interpolate_missing_values(summary, columns=["used", "interested"]):
    years = sorted(summary["year"].unique())
    languages = summary["language"].unique()

    for lang in languages:
        lang_data = summary[summary["language"] == lang]
        for col in columns:
            values_by_year = {row["year"]: row[col] for _, row in lang_data.iterrows()}

            for year in years:
                current_value = values_by_year.get(year, None)
                if current_value is None or current_value != 0.0:
                    continue  # skip non-zero or missing entries

                # Look for previous and next non-zero values
                prev = next_ = None
                for y in reversed(years[:years.index(year)]):
                    val = values_by_year.get(y)
                    if val is not None and val > 0:
                        prev = (y, val)
                        break
                for y in years[years.index(year)+1:]:
                    val = values_by_year.get(y)
                    if val is not None and val > 0:
                        next_ = (y, val)
                        break

                if prev and next_:
                    y0, v0 = prev
                    y1, v1 = next_
                    interp = v0 + (v1 - v0) * (year - y0) / (y1 - y0)
                elif prev:
                    interp = prev[1]
                elif next_:
                    interp = next_[1]
                else:
                    interp = 0.0  # no data to infer from

                # Update value in summary DataFrame
                summary.loc[
                    (summary["language"] == lang) & (summary["year"] == year), col
                ] = interp

    return summary


Next problem is the missing language years. Here the startegy is to use the previous year data to fill the gaps

In [223]:
def fill_missing_language_years(summary, years, all_languages):
    idx = pd.MultiIndex.from_product(
        [all_languages, years],
        names=["language", "year"]
    )

    df = (
        summary
        .set_index(["language", "year"])
        .reindex(idx)
        .groupby(level="language")
        .ffill() 
        .reset_index()
    )

    return df.dropna(subset=summary.columns.difference(["language","year"]))

In [224]:
def create_filtered_aggregate_dataset(df_filtered, country = None, age = None):
    total_counts = (
        df_filtered.groupby(["year", "discriminator"])
        .size()
        .unstack(fill_value=0)
        .rename(columns={"used": "total_used", "interested": "total_interested"})
    )

    language_counts = (
        df_filtered.groupby(["year", "language", "discriminator"])
        .size()
        .unstack(fill_value=0)
    )

    # Rename if columns exist
    rename_map = {}
    if "used" in language_counts.columns:
        rename_map["used"] = "used_count"
    if "interested" in language_counts.columns:
        rename_map["interested"] = "interested_count"

    language_counts = language_counts.rename(columns=rename_map)

    summary = language_counts.reset_index()
    summary = summary.merge(total_counts.reset_index(), on="year", how="left")
    summary["used"] = summary.get("used_count", 0) / summary.get("total_used", 1)
    summary["interested"] = summary.get("interested_count", 0) / summary.get("total_interested", 1)
    
    # Fill the missing language years
    all_years = sorted(df_filtered["year"].unique())
    all_languages = df_filtered["language"].unique()    
    # Here we interpolate the missing vlaues
    summary = interpolate_missing_values(summary)
    
    summary = fill_missing_language_years(summary, all_years, all_languages)


    avg_salary = (
        df_filtered.dropna(subset=["salary"])
        .groupby(["year", "language"])["salary"]
        .mean()
        .reset_index(name="avg_salary")
    )

    language_props = (
        df_filtered[["language", "execution_model", "memory_management"]]
        .drop_duplicates()
        .dropna()
        .set_index("language")
    )

    summary = summary.merge(avg_salary, on=["year", "language"], how="left")
    summary = summary.merge(language_props, on="language", how="left")

    summary = summary[[
        "year", "language", "avg_salary", "execution_model", "memory_management", "used", "interested"
    ]].sort_values(["year", "language"]).reset_index(drop=True)

    summary = summary.fillna(-1)

    # Response JSON
    result = {
        "country": country,
        "age": age,
        "max_used": summary["used"].max(),
        "max_interested": summary["interested"].max(),
        "years": {}
    }

    for year, group in summary.groupby("year"):
        result["years"][int(year)] = group.drop(columns=["year"]).to_dict(orient="records")

    
    filter_str = ''
    filter_str += f"_{country.replace(' ', '_')}" if country else ''
    filter_str += f"_{age.lower().replace(' ', '_')}" if age else ''
        
    output_path = f'../public/data/aggregate{filter_str}.json'
    with open(output_path, "w") as f:
        json.dump(result, f, indent=2)
        

create_filtered_aggregate_dataset(df_languages_all)

## Pre-filered

Now we create 203 different pre-filtered json files

In [214]:
countries = [
    "United States", "India", "Germany", "United Kingdom", "Canada", "France", "Brazil",
    "Poland", "Netherlands", "Australia", "Italy", "Spain", "Russia", "Sweden",
    "Switzerland", "Turkey", "Ukraine", "Austria", "Israel", "Pakistan",
    "Czech Republic", "Belgium", "Romania", "Iran", "China", "Portugal",
    "Greece", "Finland", "Norway"
]

age_groups = [
    "25-34 years old", "35-44 years old", "Under 18 years old", "18-24 years old",
    "45-54 years old", "55-64 years old", "65 years or older"
]

for country in countries:
    filtered_df = df_languages_all[
            (df_languages_all["country"] == country)]
    create_filtered_aggregate_dataset(filtered_df, country=country)
    
for age in age_groups:
    filtered_df = df_languages_all[
            (df_languages_all["age"] == age)]
    create_filtered_aggregate_dataset(filtered_df, age=age)

for country in countries:
    for age in age_groups:
        filtered_df = df_languages_all[
            (df_languages_all["country"] == country) &
            (df_languages_all["age"] == age)
        ]
        if not filtered_df.empty:
            filter_str = f"_{country}_{age.lower().replace(' ', '_')}"
            create_filtered_aggregate_dataset(filtered_df, country=country, age=age)